
<h1><center><font size=10> Generative AI for NLP Program</center></font></h1>
<h1><center> Project </center></h1>

# **GA-NLP Project: Financial Product Complaint Classification and Summarization**

## **Business Context**

### **Description**
*In the modern financial industry, customer complaints play a crucial role in identifying areas where financial institutions can improve their services. Effectively categorizing these complaints into specific product categories, such as credit reports, student loans, or money transfers, is essential for addressing customer concerns promptly by routing the tickets to relevant personnel. Leveraging Generative AI for text classification can help financial institutions better understand customer grievances and respond more efficiently. Apart from this, a summary of the customer complaint helps the support personnel quickly grasp the gist of the grievance*

### **Objective**
*The primary goal of this project is to utilize Generative AI techniques to improve the classification and summarization of customer complaints in the financial sector.
Specifically, the project will focus on:*

1. **Text-to-Label Classification:** *Implementing Zero-shot and Few-shot prompting methods to accurately classify customer complaints into relevant product categories.*
2. **Text-to-Text Summarization:** *Using Zero-shot prompting to generate concise summaries of customer complaints, enabling more personalized and effective responses.*

### **Conclusion**
*Upon completing this project, you will have the capability to develop solutions for LLM-based text classification and summarization. These tools will enable financial institutions to automate the complaint handling process, leading to faster, more accurate responses to customer issues, improved customer satisfaction, and enhanced compliance with industry regulations. This project will also provide you with valuable skills and experience that can be applied to a range of real-world business challenges.*


# **Section 1 : Setting Up for Prompt Engineering with Mistral Model**

### **Install & Importing neccessary libraries**

In [1]:
!apt-get update
!apt-get install -y ninja-build cmake
!pip install ipywidgets --upgrade

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
import torch

In [3]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [4]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

In [5]:
!pip install evaluate
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 131.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 111.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 82.8 MB/s eta 0:00:00


In [6]:
!pip freeze > requirement.txt

In [7]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate

# from google.colab import drive
import locale

### **Question 1: Importing Libaries and Mistral Model (3 Marks)**

- For the Mistral Model name or path and model basename, refer to the **Week 3 Additional Content: Prompt Engineering Fundamentals**
- Code Notebook: Self-Consistency and Tree-of-Thought Prompting with Llama 2 and Mistral.





https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf

In [8]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [9]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF" 
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [10]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

In [11]:
lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /home/andy/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_mode

# **Section 2: Text to Label generation**

### **Question 2: Zero-Shot Prompting for Text Classification (5 Marks)**

##### **Q2.1: Define the Prompt Template, System Message, generate_prompt** **(2 Marks)**

- Define a **system message** as a string and assign it to the variable system_message to generate product class.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [12]:
system_message = "You are a helpful assistant that classifies user complaints into one of the following product categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, or debt_collection. Respond only with the product category."


In [13]:
zero_shot_prompt_template = """[System Message]\n{system_message}\n\n[User Complaint]\n{user_input}\n\n[Response]"""


In [14]:
def generate_prompt(system_message,user_input):
    prompt = zero_shot_prompt_template.format(system_message=system_message, user_input=user_input)
    return prompt

In [15]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )
    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [16]:
# Load a CSV File containing Dataset of 500 products, narrative and summary (summary of narrative)
data = pd.read_csv("Complaints_classification.csv")


In [17]:
# Randomly select 30 rows
new_data = data.sample(n=30, random_state=40)

In [18]:
new_data

,product,narrative,summary
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...
253,credit_reporting,block except otherwise provided section consum...,The text pertains to the stipulations and oper...
42,credit_reporting,open account acct opened balance account acct ...,The input is about various accounts being open...
369,credit_reporting,except otherwise provided section consumer rep...,This legal text details rules and regulations ...
26,mortgages_and_loans,trying close loan month provided income paymen...,The individual has been attempting to close a ...
377,credit_reporting,except otherwise provided section consumer rep...,This legislation requires that a consumer repo...
238,credit_reporting,block except otherwise provided section consum...,The section explains the rules and procedures ...
374,credit_reporting,except otherwise provided section consumer rep...,The input text seems to discuss a law provisio...


##### **Q2.2: Create a new column in the DataFrame called 'mistral_response' and populate it with responses generated by applying the 'generate_mistral_response' function to each 'narrative' in the DataFrame and prepare the mistral_response_cleaned column using extract_category function** **(1 Marks)**

In [19]:
# example - new_data['mistral_response'] = new_data['narrative'].apply(lambda x:______ )
#new_data['mistral_response'] = new_data['narrative'].apply(lambda x: "_____")

new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.30 ms /     6 runs   (    0.22 ms per token,  4629.63 tokens per second)
llama_print_timings: prompt eval time =     408.95 ms /   284 tokens (    1.44 ms per token,   694.45 tokens per second)
llama_print_timings:        eval time =     156.06 ms /     5 runs   (   31.21 ms per token,    32.04 tokens per second)
llama_print_timings:       total time =     576.36 ms /   289 tokens
Llama.generate: prefix-match hit



credit_card



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.37 ms /     7 runs   (    0.20 ms per token,  5124.45 tokens per second)
llama_print_timings: prompt eval time =     431.81 ms /   505 tokens (    0.86 ms per token,  1169.50 tokens per second)
llama_print_timings:        eval time =     187.49 ms /     6 runs   (   31.25 ms per token,    32.00 tokens per second)
llama_print_timings:       total time =     632.48 ms /   511 tokens
Llama.generate: prefix-match hit



credit_reporting



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.22 ms /     6 runs   (    0.20 ms per token,  4909.98 tokens per second)
llama_print_timings: prompt eval time =     221.97 ms /   182 tokens (    1.22 ms per token,   819.95 tokens per second)
llama_print_timings:        eval time =     160.22 ms /     5 runs   (   32.04 ms per token,    31.21 tokens per second)
llama_print_timings:       total time =     392.25 ms /   187 tokens
Llama.generate: prefix-match hit


...


credit_reporting



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.38 ms /     7 runs   (    0.20 ms per token,  5083.51 tokens per second)
llama_print_timings: prompt eval time =     433.55 ms /   505 tokens (    0.86 ms per token,  1164.81 tokens per second)
llama_print_timings:        eval time =     191.54 ms /     6 runs   (   31.92 ms per token,    31.32 tokens per second)
llama_print_timings:       total time =     636.36 ms /   511 tokens
Llama.generate: prefix-match hit



credit_reporting



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.39 ms /     7 runs   (    0.20 ms per token,  5032.35 tokens per second)
llama_print_timings: prompt eval time =     192.09 ms /    95 tokens (    2.02 ms per token,   494.57 tokens per second)
llama_print_timings:        eval time =     182.99 ms /     6 runs   (   30.50 ms per token,    32.79 tokens per second)
llama_print_timings:       total time =     385.91 ms /   101 tokens



credit_reporting


In [20]:
new_data['mistral_response']

167            \ncredit_card
169       \ncredit_reporting
461        \ndebt_collection
253       \ncredit_reporting
42          \nretail_banking
369       \ncredit_reporting
26     \nmortgages_and_loans
377       \ncredit_reporting
238       \ncredit_reporting
374       \ncredit_reporting
140       \ncredit_reporting
175       \ncredit_reporting
388       \ncredit_reporting
62     \nmortgages_and_loans
256       \ncredit_reporting
332       \ncredit_reporting
386       \ncredit_reporting
56             \ncredit_card
157       \ncredit_reporting
48     \nmortgages_and_loans
163       \ncredit_reporting
9         \ncredit_reporting
441       \ncredit_reporting
102       \ncredit_reporting
0           \nretail_banking
364       \ncredit_reporting
139       \ncredit_reporting
150       \ncredit_reporting
173       \ncredit_reporting
463       \ncredit_reporting
Name: mistral_response, dtype: object

In [21]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

In [22]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x:extract_category(x) )

In [23]:
new_data.head()

,product,narrative,summary,mistral_response,mistral_response_cleaned
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...,\ncredit_card,credit_card
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...,\ncredit_reporting,credit_reporting
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...,\ndebt_collection,debt_collection
253,credit_reporting,block except otherwise provided section consum...,The text pertains to the stipulations and oper...,\ncredit_reporting,credit_reporting
42,credit_reporting,open account acct opened balance account acct ...,The input is about various accounts being open...,\nretail_banking,retail_banking


##### **Q2.3: Calculate the F1 score** **(1 Marks)**

In [24]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [25]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f2 = f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='micro')
print(f'F1 Score: {f2}')

F1 Score: 0.8333333333333334


##### **Q2.4: Explain the difference in F1 scores between mistral_response and mistral_response_cleaned.** **(1 Marks)**

#### mistral_response_cleaned has a higher F1 score because it extracts only the predicted category, removing extra text from the full mistral_response.


### **Question 3: Few-Shot Prompting for Text Classification (7 Marks)**

##### **Q3.1: Prepare examples for a few-shot prompt, formulate the prompt, and generate the Mistral response. (5 Marks)**

**Generate a set of gold examples by randomly selecting 10 instances of user_input and assistant_output from dataset ensuring a balanced representation with 2 examples from each class.**

In [26]:
import json
review_1 = data[data['product'] == 'credit_card']
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

# Create the training set by excluding examples
gold_examples_df = data.drop(index=examples_df.index)

# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(examples_json)[0])

# Print the shapes of the datasets
print("Examples Set Shape:", examples_df.shape)
print("Gold Examples Shape:", gold_examples_df.shape)

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}
Examples Set Shape: (10, 3)
Gold Examples Shape: (490, 3)


- Define your **system_message**.
- Define **first_turn_template**, **example_template** and **prediction template**
- **create few shot prompt** using gold examples and system_message
- Randomly select 30 rows from test_df as test_data
- Create **mistral_response** with **mistral_response_cleaned** columns for this

In [27]:
system_message = "You are a helpful assistant that classifies user complaints into one of the following product categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, or debt_collection. Respond only with the product category."


In [28]:

first_turn_template = """[System Message]
{system_message}

[User Complaint]
{user_input}

[Assistant Response]
{assistant_output}

"""

# Template for subsequent examples
examples_template = """[User Complaint]
{user_input}

[Assistant Response]
{assistant_output}

"""

prediction_template = """[User Complaint]
{user_input}

[Assistant Response]"""

In [29]:
def create_few_shot_prompt(system_message, examples_df):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    columns_to_select = ['narrative', 'product']
    examples = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples)):
        user_input_example = example['narrative']
        assistant_output_example = example['product']

        if idx == 0:
            few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )
        else:
            few_shot_prompt += examples_template.format(
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )

    return few_shot_prompt

In [30]:
few_shot_prompt = create_few_shot_prompt(system_message, examples_df)


In [31]:
print(few_shot_prompt)

[System Message]
You are a helpful assistant that classifies user complaints into one of the following product categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, or debt_collection. Respond only with the product category.

[User Complaint]
called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee

[Assistant Response]
credit_card

[User Complaint]
dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute making purchase credit card usaa two separate transaction merchant purchased trip cancelled told apply refund apply refund however company give refund back filed dispute credit agency help dispute account follow dispute policy state service provided customer right receive full ref

In [32]:
def generate_prompt(few_shot_prompt,new_review):
    prompt =  few_shot_prompt + prediction_template.format(user_input=new_review)
    return prompt

In [33]:
def generate_mistral_response(support_ticket_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,support_ticket_text)

    # Define the Llama model along with its parameters for generating a response
  # Call the Llama model to generate a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [34]:
# Randomly select 50 rows from gold_examples
new_data = gold_examples_df.sample(n=50, random_state=40)

In [35]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
#new_data['mistral_response'] = new_data['narrative'].apply(lambda x: "_____")
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))


Llama.generate: prefix-match hit

llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.42 ms /     7 runs   (    0.20 ms per token,  4936.53 tokens per second)
llama_print_timings: prompt eval time =     689.01 ms /   719 tokens (    0.96 ms per token,  1043.53 tokens per second)
llama_print_timings:        eval time =     193.70 ms /     6 runs   (   32.28 ms per token,    30.98 tokens per second)
llama_print_timings:       total time =     898.20 ms /   725 tokens
Llama.generate: prefix-match hit


 mortgage_and_loans



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       0.89 ms /     5 runs   (    0.18 ms per token,  5636.98 tokens per second)
llama_print_timings: prompt eval time =     386.02 ms /   434 tokens (    0.89 ms per token,  1124.29 tokens per second)
llama_print_timings:        eval time =     125.88 ms /     4 runs   (   31.47 ms per token,    31.78 tokens per second)
llama_print_timings:       total time =     520.08 ms /   438 tokens
Llama.generate: prefix-match hit


 credit_reporting



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       1.42 ms /     7 runs   (    0.20 ms per token,  4919.18 tokens per second)
llama_print_timings: prompt eval time =     163.30 ms /    19 tokens (    8.59 ms per token,   116.35 tokens per second)
llama_print_timings:        eval time =     183.32 ms /     6 runs   (   30.55 ms per token,    32.73 tokens per second)
llama_print_timings:       total time =     359.85 ms /    25 tokens
Llama.generate: prefix-match hit


...

 mortgage_and_loans



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       0.89 ms /     5 runs   (    0.18 ms per token,  5643.34 tokens per second)
llama_print_timings: prompt eval time =     382.15 ms /   422 tokens (    0.91 ms per token,  1104.28 tokens per second)
llama_print_timings:        eval time =     123.46 ms /     4 runs   (   30.86 ms per token,    32.40 tokens per second)
llama_print_timings:       total time =     513.68 ms /   426 tokens
Llama.generate: prefix-match hit


 credit_reporting



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =       0.93 ms /     5 runs   (    0.18 ms per token,  5405.41 tokens per second)
llama_print_timings: prompt eval time =     386.11 ms /   434 tokens (    0.89 ms per token,  1124.04 tokens per second)
llama_print_timings:        eval time =     125.08 ms /     4 runs   (   31.27 ms per token,    31.98 tokens per second)
llama_print_timings:       total time =     519.08 ms /   438 tokens


 credit_reporting


In [36]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
#new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: "_____")
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: extract_category(x))


##### **Q3.2: Calculate the F1 score** **(1 Marks)**

In [37]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 = f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='micro')
print(f'F1 Score: {f1}')
print(f'F1 Score: {f1}')

F1 Score: 0.84
F1 Score: 0.84


##### **Q3.3: Share your observations on the few-shot and zero-shot prompt techniques. (1 Marks)**

############ Few-shot prompting improves accuracy by showing examples, while zero-shot relies only on instructions and is simpler but often less precise.

# **Section 3: Text to Text generation**

### **Question 4: Zero-Shot Prompting for Text Summarization (5 Marks)**

##### **Q4.1: Define the Prompt Template, System Message, generate prompt and model response** **(3 Marks)**


- Define a **system message** as a string and assign it to the variable system_message to generate summary of narrative in data.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [38]:
system_message = "You are a helpful assistant that generates a concise summary of the given complaint narrative."


In [39]:
zero_shot_prompt_template = """[System Message]
{system_message}

[User Complaint]
{user_input}

[Summary]
"""
def generate_prompt(system_message,user_input):
    prompt=zero_shot_prompt_template.format(system_message=system_message,user_input =user_input)
    return prompt

def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )


    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

##### **Q4.2: Generate mistral_response column containing LLM generated summaries** **(1 Marks)**

In [40]:
# Randomly select 30 rows
gold_examples = data.sample(30, random_state=40)

In [41]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
gold_examples['mistral_response'] = gold_examples['narrative'].apply(lambda x: generate_mistral_response(x))


Llama.generate: prefix-match hit

llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =      33.14 ms /   163 runs   (    0.20 ms per token,  4918.68 tokens per second)
llama_print_timings: prompt eval time =     248.31 ms /   247 tokens (    1.01 ms per token,   994.71 tokens per second)
llama_print_timings:        eval time =    5066.79 ms /   162 runs   (   31.28 ms per token,    31.97 tokens per second)
llama_print_timings:       total time =    5602.62 ms /   409 tokens
Llama.generate: prefix-match hit


The user is reporting a fraudulent charge on their Capital One checking account, which was immediately canceled. They disputed the charge and were informed that provisional credit would be issued pending determination of the claim. However, they received a form letter denying the claim despite never losing possession of their debit card or authorizing someone else to use it. The user believes their card may have been intercepted and fraudulently activated for unauthorized purchases. They have contacted Capital One multiple times to discuss the original claim and make a determination, but feel that malfeasance is occurring as they were sent a replacement debit card without authorization and had credit purchased on their account without consent. The user has also reported this incident to local police department's financial services division regarding fraudulent activity.



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =      34.99 ms /   169 runs   (    0.21 ms per token,  4830.37 tokens per second)
llama_print_timings: prompt eval time =     439.50 ms /   506 tokens (    0.87 ms per token,  1151.32 tokens per second)
llama_print_timings:        eval time =    5295.43 ms /   168 runs   (   31.52 ms per token,    31.73 tokens per second)
llama_print_timings:       total time =    6043.08 ms /   674 tokens
Llama.generate: prefix-match hit


The Consumer Reporting Agency (CRA) is required to block reporting information if a consumer identifies information that resulted from alleged identity theft. The CRA should promptly notify the furnisher of this information and may decline or rescind the block upon reasonable determination, providing evidence whether the consumer was aware of the good service resulting in blocked transactions. Consumers have the right to be notified promptly if their information is reinstated. If a reseller file contains such information, they must provide notice to the consumer. The CRA may apply exceptions for verification companies and check services acting on behalf of consumers. Law enforcement agencies cannot access blocked information without proper authorization or verifiable proof from the original consumer contract. Unverified accounts listed in reports must be removed if the consumer is unable to provide copy verifiable proof.



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =      34.97 ms /   166 runs   (    0.21 ms per token,  4746.65 tokens per second)
llama_print_timings: prompt eval time =     224.81 ms /   183 tokens (    1.23 ms per token,   814.03 tokens per second)
llama_print_timings:        eval time =    5283.10 ms /   165 runs   (   32.02 ms per token,    31.23 tokens per second)
llama_print_timings:       total time =    5814.09 ms /   348 tokens
Llama.generate: prefix-match hit


...

The user is a victim of identity theft and has discovered unauthorized transactions listed on their Equifax credit report. They have been trying to resolve the issue for an extended period, but it has proven to be complicated and time-consuming. The user fears that they may have been involved in the Equifax data breach and is concerned about potential fraudulent debts hindering their ability to apply for credit or even get a first-time credit card. They are currently reviewing their credit file carefully, considering seeking legal advice from an attorney specializing in identity theft cases, and are eligible for victim compensation. The user has noticed a disputed account on their Equifax report that they have not been able to resolve, causing financial and emotional stress. Additionally, the negative information reported may be hindering their ability to get a job due to adverse actions based on inaccurate credit reporting.



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =      33.90 ms /   169 runs   (    0.20 ms per token,  4985.99 tokens per second)
llama_print_timings: prompt eval time =     438.92 ms /   506 tokens (    0.87 ms per token,  1152.84 tokens per second)
llama_print_timings:        eval time =    5243.94 ms /   168 runs   (   31.21 ms per token,    32.04 tokens per second)
llama_print_timings:       total time =    5982.23 ms /   674 tokens
Llama.generate: prefix-match hit


The Consumer Reporting Agency (CRA) is required to block reporting information if a consumer identifies information that resulted from alleged identity theft. The CRA should promptly notify the furnisher of this information and may decline or rescind the block upon reasonable determination, providing evidence whether the consumer was aware of the good service resulting in blocked transactions. Consumers have the right to be notified promptly if their information is reinstated. If a reseller file contains such information, they must provide notice to the consumer. The CRA may apply exceptions for verification companies and check services acting on behalf of consumers. Law enforcement agencies cannot access blocked information without proper authorization or verifiable proof from the original consumer contract. Unverified accounts listed in reports must be removed if the consumer is unable to provide copy verifiable proof.



llama_print_timings:        load time =     409.10 ms
llama_print_timings:      sample time =      23.75 ms /   121 runs   (    0.20 ms per token,  5094.95 tokens per second)
llama_print_timings: prompt eval time =     189.77 ms /    96 tokens (    1.98 ms per token,   505.89 tokens per second)
llama_print_timings:        eval time =    3660.47 ms /   120 runs   (   30.50 ms per token,    32.78 tokens per second)
llama_print_timings:       total time =    4060.06 ms /   216 tokens


The user's mother called to remove an authorized user from her credit card who had never used the line. However, this action negatively affected the user's credit score. The user was able to contact Experian and dispute the change but is eagerly waiting for the reflection of the correction in their score. They need a good credit score to refinance their home as their husband has lost his job and they are barely making ends meet with a nurse's salary. Despite trying to dispute online, the user was unable to get through to Experian by phone to further discuss the issue.


##### **Q4.3: Evaluate bert score** **(1 Marks)**

In [42]:
def evaluate_score(test_data, scorer, bert_score=False):

    """
    Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        scorer (function): Scorer function used to compute the ROUGE score or the
                           BERTScore
        bert_score (boolean): A flag variable that indicates if BERTScore should
                              be used as the metric.

    Output:
        score (float): BERTScore or ROUGE score computed by comparing model predictions
                       with ground truth
    """

    model_predictions = test_data['mistral_response'].tolist()
    ground_truths = test_data['summary'].tolist()
    if bert_score:
        score = scorer.compute(
            predictions=model_predictions,
            references=ground_truths,
            lang="en",
            rescale_with_baseline=True
        )

        return sum(score['f1'])/len(score['f1'])
    else:
        return scorer.compute(
            predictions=model_predictions,
            references=ground_truths
        )

In [43]:
bert_scorer = evaluate.load("bertscore")

In [44]:
score = evaluate_score(
    gold_examples,   
    bert_scorer,
    bert_score=True
)


print(f'BERTScore: {score}')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.3519383544723193
